In [7]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("GPU 이름:", torch.cuda.get_device_name(0))


CUDA Available: True
GPU 개수: 1
GPU 이름: Tesla T4


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 샘플 데이터 (긍정 1, 부정 0)
data = [
    ("이 제품 사용감이 별로네요", 0),
    ("정말 좋아요! 만족합니다.", 1),
    ("별로예요. 다시는 안 살 듯", 0),
    ("매우 편리하고 좋아요!", 1),
    ("퀄리티가 너무 낮아요.", 0)
]

df = pd.DataFrame(data, columns=['review', 'label'])

# 형태소 분석 및 토큰화
okt = Okt()

def tokenize(text):
    return okt.morphs(text, stem=True)  # 원형 복원

df['tokenized'] = df['review'].apply(tokenize)

# 단어 사전 생성
vocab = {word: idx + 2 for idx, word in enumerate(set(sum(df['tokenized'].tolist(), [])))}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

# 텍스트를 인덱스로 변환
def encode_text(tokenized_sentence):
    return [vocab.get(word, 1) for word in tokenized_sentence]

df['encoded'] = df['tokenized'].apply(encode_text)

# 패딩 적용 (최대 길이 10)
MAX_LEN = 10

def pad_sequence(seq):
    return seq[:MAX_LEN] + [0] * (MAX_LEN - len(seq))

df['padded'] = df['encoded'].apply(pad_sequence)

# 데이터 분할
X = torch.tensor(df['padded'].tolist(), dtype=torch.long)
y = torch.tensor(df['label'].tolist(), dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, hidden_dim=64, output_dim=1):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        x = self.fc(hidden[-1])
        return self.sigmoid(x)

# 모델 초기화
vocab_size = len(vocab)
model = SentimentLSTM(vocab_size).cuda()

# 손실 함수 & 옵티마이저
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
BATCH_SIZE = 2
EPOCHS = 10

train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
        
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = tokenize(sentence)
    encoded = encode_text(tokenized)
    padded = pad_sequence(encoded)
    input_tensor = torch.tensor([padded], dtype=torch.long).cuda()
    
    with torch.no_grad():
        output = model(input_tensor).item()
    
    return "긍정" if output > 0.5 else "부정"

# 예제 리뷰 평가
test_sentence = "이 제품 사용감이 별로네요"
print(f"리뷰: {test_sentence} -> {predict_sentiment(model, test_sentence)}")


Epoch 1, Loss: 1.4353
Epoch 2, Loss: 1.4196
Epoch 3, Loss: 1.3952
Epoch 4, Loss: 1.3788
Epoch 5, Loss: 1.3646
Epoch 6, Loss: 1.3416
Epoch 7, Loss: 1.3175
Epoch 8, Loss: 1.2907
Epoch 9, Loss: 1.2570
Epoch 10, Loss: 1.2168
리뷰: 이 제품 사용감이 별로네요 -> 부정


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Okt

# 샘플 데이터 (비지도 학습이므로 라벨 없음)
reviews = [
    "이 제품 사용감이 별로네요",
    "정말 좋아요! 만족합니다.",
    "별로예요. 다시는 안 살 듯",
    "매우 편리하고 좋아요!",
    "퀄리티가 너무 낮아요.",
    "배송이 빠르고 좋아요!",
    "디자인이 멋있고 성능이 뛰어나요!",
    "가성비가 별로인 것 같아요."
]

# 형태소 분석 및 전처리
okt = Okt()

def tokenize(text):
    return " ".join(okt.morphs(text, stem=True))

processed_reviews = [tokenize(review) for review in reviews]

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_reviews)

# K-Means 클러스터링 (k=2로 긍/부정 예상)
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# 클러스터 할당 결과 출력
for i, review in enumerate(reviews):
    print(f"리뷰: {review} -> 클러스터 {kmeans.labels_[i]}")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# LDA를 위한 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(processed_reviews)

# LDA 모델 (2개의 주요 토픽 찾기)
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)

# 주요 단어 출력
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"토픽 {topic_idx}: ", [feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])



리뷰: 이 제품 사용감이 별로네요 -> 클러스터 0
리뷰: 정말 좋아요! 만족합니다. -> 클러스터 2
리뷰: 별로예요. 다시는 안 살 듯 -> 클러스터 0
리뷰: 매우 편리하고 좋아요! -> 클러스터 1
리뷰: 퀄리티가 너무 낮아요. -> 클러스터 0
리뷰: 배송이 빠르고 좋아요! -> 클러스터 1
리뷰: 디자인이 멋있고 성능이 뛰어나요! -> 클러스터 0
리뷰: 가성비가 별로인 것 같아요. -> 클러스터 0
토픽 0:  ['좋다', '별로', '정말', '만족하다', '배송', '빠르다', '가성', '같다', '다시다', '예요']
토픽 1:  ['성능', '뛰어나다', '디자인', '멋있다', '퀄리티', '너무', '낮다', '매우', '편리하다', '좋다']
토픽 2:  ['별로', '용감', '제품', '좋다', '예요', '다시다', '가성', '같다', '배송', '빠르다']


In [ ]:
https://wonhwa.tistory.com/35